In [16]:
from datasets import load_metric

In [14]:
def form_corpus_label_from_manual_anno(text):
    """
    Modify the annotated label, such that model can use it as Tags
    """
    text_1 = text.split(' ')
    new_text = []
    new_id = []
    for word in text_1:
        if word == '[' or word == ']' or word == ',' or word == '"' or word == '\n' or word == ' ' or word == '':
            pass
        else:
            n_w = ''
            for i in word:
                if i == '[' or i == ']' or i == ',' or i == '"' or i == '\n' or i == '\'':
                    pass
                else:
                    n_w = n_w + i
            if n_w != '':
                new_text.append(n_w)
    return ( new_text)


i = 0
text_corpus = {'words':[],
              'ner' : []}
with open("annotated_text.txt", 'r') as f:
    for line in f.readlines():
        if i % 3 == 0:
            text = line
            text_1 = text.split(" ")[1:]
            if text_1[-1] == '\n':
                text_1 = text_1[0:-1]
            text_corpus['words'].append(text_1)
        elif i % 3 == 1:
            label = line
            new_label  = form_corpus_label_from_manual_anno(label)
            text_corpus['ner'].append(new_label)
        i = i + 1

In [15]:
# CHat-GPT generated NER Tags from the corpus
chat_gpt_generated_ner_tags = [
[ "B-MISC", "I-MISC", "I-MISC", "O", "O", "O", "O","O", "O", "O", "O"],

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ], 

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], 

["B-PER", "I-PER", "I-PER", "O", "B-PER", "I-PER", "O", "B-PER", "I-PER", "O", "O", "O", "O"],

["B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O"], 

["B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O"], 

["O", "O", "B-LOC", "I-LOC", "O", "B-LOC", "O", "O", "O", "O", "B-PER", "I-PER", "I-PER", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "B-PER", "I-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] ,

 ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O",  "O"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["O", "O", "MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] ,

["MISC", "MISC", "MISC", "MISC", "O", "O", "MISC", "MISC", "MISC", "MISC"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG","O"] ,

["O", "O", "O", "B-LOC", "I-LOC", "O", "O", "O", "O","O"] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ], 

["O", "O", "O","B-MISC", "B-ORG", "O","B-MISC", "O", "O", "O", "O", "O", "O", "O","O", "O", "O", "O", "O", "O" ] ,

["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"] 
    
]

In [18]:
for i in range(len(text_corpus['ner'])):
    (j, k) = (len(text_corpus['ner'][i]), len(chat_gpt_generated_ner_tags[i]))
    if j > k:
        for _ in range(j - k):
            chat_gpt_generated_ner_tags[i].append(0)
    elif j < k :
        chat_gpt_generated_ner_tags[i] = chat_gpt_generated_ner_tags[i][0 : k - (k - j)]  

In [19]:
metric = load_metric("seqeval")
results = metric.compute(predictions=chat_gpt_generated_ner_tags, references=text_corpus['ner'])
# Unpack nested dictionaries
final_results = {}
for key, value in results.items():
    if isinstance(value, dict):
        for n, v in value.items():
            final_results[f"{key}_{n}"] = v
    else:
        final_results[key] = value

/home/user/anaconda3/envs/env1/lib/python3.12/site-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/user/anaconda3/envs/env1/lib/python3.12/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/user/anaconda3/envs/env1/lib/python3.12/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/user/anaconda3/envs/env1/lib/python3.12/si

In [20]:
final_results

{'ISC_precision': 0.0,
 'ISC_recall': 0.0,
 'ISC_f1': 0.0,
 'ISC_number': 0,
 'LOC_precision': 0.3333333333333333,
 'LOC_recall': 0.25,
 'LOC_f1': 0.28571428571428575,
 'LOC_number': 4,
 'MISC_precision': 0.0,
 'MISC_recall': 0.0,
 'MISC_f1': 0.0,
 'MISC_number': 20,
 'ORG_precision': 0.0,
 'ORG_recall': 0.0,
 'ORG_f1': 0.0,
 'ORG_number': 2,
 'PER_precision': 0.5833333333333334,
 'PER_recall': 0.4375,
 'PER_f1': 0.5,
 'PER_number': 16,
 'overall_precision': 0.36363636363636365,
 'overall_recall': 0.19047619047619047,
 'overall_f1': 0.25,
 'overall_accuracy': 0.8113695090439277}